##### This notebook demonstrates a demo of how you can deploy your first job with TrueFoundry.

---


After you complete the notebook, you will have a successful deployed a job to train a model on the iris dataser. Your jobs deployment dashboard will look like this:

![](https://files.readme.io/2f6871c-Screenshot_2022-11-16_at_11.43.31_PM.png)


## Project structure

To complete this guide, you are going to create the following **files**:

- `train.py` : contains our training code
- `requirements.txt` : contains our dependencies
- `deploy.py` contains our deployment code ( you can also use a deployment configuration for deploying using a YAML file)

Your **final file structure** is going to look like this:

```
.
├── train.py
├── deploy.py
└── requirements.txt
```

As you can see, all the following files are created in the same folder/directory


# Setup

Let's first setup all the things we need to deploy our service.

- Signup or Login on TrueFoundry
- Setup Workspace


Let's start with installing `truefoundry`.


In [ ]:
%pip install -U "truefoundry>=0.10.0,<0.11.0"

**Login into TrueFoundry**

In order to login run the cell below. Host can be found from the TrueFoundry UI as shown below like https://app.truefoundry.com

![image.png](../common/images/host.png)

In [ ]:
!tfy login --host "<Host name of TrueFoundry UI. e.g. https://company.truefoundry.cloud>"

**Select the `Workspace` in which you want to deploy your application.**

Once you run the cell below you will get a prompt to enter your Workspace FQN. Follow the docs to

**Create a Workspace**: https://docs.truefoundry.com/docs/key-concepts#creating-a-workspace

**Get Existing Workspace FQN**: https://docs.truefoundry.com/docs/key-concepts#get-workspace-fqn

In [ ]:
import click

In [ ]:
WORKSPACE_FQN = click.prompt(
    "Enter the Workspace FQN",
    type=str,
)
print(f"\nWorkspace FQN set to {WORKSPACE_FQN!r}")

# Step 1: Implement the training code

The first step is to create a job that trains a scikit learn model on iris dataset

We start with a `train.py` containing our training code and `requirements.txt` with our dependencies.

```
.
├── train.py
└── requirements.txt
```


### **`train.py`**


In [ ]:
%%writefile train.py
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# load the dataset
X, y = load_iris(as_frame=True, return_X_y=True)
X = X.rename(columns={
        "sepal length (cm)": "sepal_length",
        "sepal width (cm)": "sepal_width",
        "petal length (cm)": "petal_length",
        "petal width (cm)": "petal_width",
})

# NOTE:- You can pass these configurations via command line
# arguments, config file, environment variables.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Initialize the model
clf = LogisticRegression(solver="liblinear")
# Fit the model
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print(classification_report(y_true=y_test, y_pred=preds))

Click on this [link](https://docs.truefoundry.com/v0.1.1/recipes/training-a-scikit-learn-model) to understand the **`app.py`**:


### **`requirements.txt`**


In [ ]:
%%writefile requirements.txt
pandas==1.5.3
numpy==1.23.2
scikit-learn==1.5.0

# Step 2: Deploying as a Job

You can deploy services on TrueFoundry programmatically via our **Python SDK**.

Create the `deploy.py`, after which our file structure will look like this:

**File Structure**

```Text
.
├── train.py
├── deploy.py
└── requirements.txt
```

### **`deploy.py`**


In [ ]:
%%writefile deploy.py
import argparse
import logging

from truefoundry.deploy import Build, Job, PythonBuild, LocalSource

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(name)s] %(levelname)-8s %(message)s"
)

parser = argparse.ArgumentParser()
parser.add_argument("--workspace_fqn", required=True, type=str)
args = parser.parse_args()

job = Job(
    name="iris-train-job",
    image=Build(
        build_source=LocalSource(local_build=False),
        build_spec=PythonBuild(
            python_version="3.11",
            command="python train.py",
            requirements_path="requirements.txt",
        )
    )
)
job.deploy(workspace_fqn=args.workspace_fqn, wait=False)

Now to deploy our Job run the command below


In [ ]:
!python deploy.py --workspace_fqn $WORKSPACE_FQN